In [1]:
import pygame as pg
import pymunk # simulates in C -> fast 
import numpy as np
import random

# random.seed(42)

pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# docs
# pygame for visualization: https://www.pygame.org/docs/
# pymunk as physics engine: https://www.pymunk.org/en/latest/overview.html

# references
# simulator with pymunk: https://www.youtube.com/watch?v=yJK5J8a7NFs (used this one a lot)
# pygame city builder: https://www.youtube.com/watch?v=wI_pvfwcPgQ
# james allen - agent based modelling: https://www.youtube.com/watch?v=RglNX4c_dfc

In [3]:
# report

# explain our particles (persons): 
#    - shape (with attributes)
#    - velocity
#    - collision_radius

In [4]:
# constants
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_GREY = (70, 84, 105)
RED = (252, 3, 65)

FPS = 60

In [5]:
# Playground
a = np.ndarray((10,10))
a[0]

array([6.89896366e-310, 6.89896366e-310, 4.67598527e-310, 4.67598527e-310,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000])

In [6]:
class Person():
    def __init__(self, world, x_init, y_init, collision_radius=10):
        # setup person as a pymunk circle particle
        self.x = x_init
        self.y = y_init
        self.collision_radius = collision_radius
        self.body = pymunk.Body()
        self.body.position = x_init, y_init
        
        # initial velocity
        self.body.velocity = random.uniform(-100, 100), random.uniform(-100, 100)
        # self.body.velocity = 50, 50
        # self.body.velocity = self.x/10, self.y/10
        
        self.shape = pymunk.Circle(self.body, self.collision_radius)
        self.shape.density = 1
        self.shape.elasticity = 1
        
        # setup attributes
        # subject
        # age (bimodal distribution: https://stackoverflow.com/questions/60721826/how-can-i-generate-n-random-values-from-a-bimodal-distribution-in-python)
        # vaccinated
        # planned_path
        # etc.
        
        # add the person to the world
        world.add(self.body, self.shape)
    
    def update_velocity(self):
        # make dependent on current velocity (self.body.velocity) and planned path
        pass
    
    def draw(self, screen):
        x, y = self.body.position
        discrete_position = (int(x), int(y))
        pg.draw.circle(screen, BLACK, discrete_position, self.collision_radius)

In [7]:
class Wall():
    def __init__(self, world, start_pos, end_pos, thickness=3):
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.thickness = thickness
        self.body = pymunk.Body(body_type=pymunk.Body.STATIC) # static body
        self.shape = pymunk.Segment(self.body, start_pos, end_pos, radius=thickness) # people might glitch through if not big enough
        self.shape.elasticity = 1
        world.add(self.body, self.shape)
    
    def draw(self, screen):
        pg.draw.line(screen, RED, self.start_pos, self.end_pos, self.thickness)

In [8]:
class Train():
    def __init__(self, world, movement_start_pos, movement_stop_pos, movement_end_pos, wall_thickness=3):
        self.movement_start_pos = movement_start_pos
        self.movement_stop_pos = movement_stop_pos # position of the trainstation (where the train stops)
        self.movement_end_pos = movement_end_pos
        self.wall_thickness = wall_thickness
        
        self.moving = True
        self.stopped_at_station = False
        
        x, y = movement_start_pos        
        self.body = pymunk.Body(mass=100, body_type=pymunk.Body.KINEMATIC)
        
        # self.shape = pymunk.Poly(self.body)
        self.shape1 = pymunk.Segment(self.body, (x, y), (x+20, y), radius=self.wall_thickness) # top-right
        self.shape2 = pymunk.Segment(self.body, (x, y), (x, y+80), radius=self.wall_thickness) # top-right
        self.shape3 = pymunk.Segment(self.body, (x, y+80), (x+20, y+80), radius=self.wall_thickness) # top-right
        self.shape4 = pymunk.Segment(self.body, (x+20, y), (x+20, y+80), radius=self.wall_thickness) # top-right
                
        self.shape1.elasticity = 1
        self.shape2.elasticity = 1
        self.shape3.elasticity = 1
        self.shape4.elasticity = 1
        
        self.body.position = x, y
        self.body.velocity = (-1.1, 30)
        
        # add the train to the world
        world.add(self.body, self.shape1, self.shape2, self.shape3, self.shape4)
        
    def update_velocity(self):
        # movement/position depends on current timestep
        pass
    
    def draw(self, screen):
        x_float, y_float = self.body.position
        x, y = int(x_float), int(y_float)
        pg.draw.line(screen, BLACK, (x+70, y), (x+90, y), self.wall_thickness) # top-right
        pg.draw.line(screen, BLACK, (x+70, y), (x+70, y+80), self.wall_thickness) # top-down(left)
        pg.draw.line(screen, BLACK, (x+70, y+80), (x+90, y+80), self.wall_thickness) # bot-right
        pg.draw.line(screen, BLACK, (x+90, y), (x+90, y+80), self.wall_thickness) # top-down(right)

In [9]:
class VirusSim():
    
    # _create_tile here
    
    def __init__(self, n_people, screen_size, world_size):
        # game setup
        self.screen_size = screen_size
        self.screen = pg.display.set_mode((screen_size, screen_size))
        self.width, self.height = self.screen.get_size()
        self.clock = pg.time.Clock()
        self.running = True
        
        # logo and caption
        logo = pg.image.load('images/virus_logo.png')
        pg.display.set_icon(logo)
        pg.display.set_caption('Virus Sim')
        
        # setup world (one tile is a 10px by 10px square)
        self.world = pymunk.Space()
        self.world_size = world_size
        
        # add screen borders as walls
        self.screen_borders = [
            Wall(world=self.world, start_pos=(0, 0), end_pos=(800, 0)),
            Wall(world=self.world, start_pos=(0, 0), end_pos=(0, 800)),
            Wall(world=self.world, start_pos=(0, 800), end_pos=(800, 800)),
            Wall(world=self.world, start_pos=(800, 0), end_pos=(800, 800))]
        
        self.buildings = [
            self._create_tile(origin_pos=(630,490), tile_type='building_1'),
            self._create_tile(origin_pos=(620,450), tile_type='building_2'),
            self._create_tile(origin_pos=(700,530), tile_type='building_3'),
            self._create_tile(origin_pos=(690,430), tile_type='building_4'),
            self._create_tile(origin_pos=(700,310), tile_type='building_5'),
            self._create_tile(origin_pos=(630,310), tile_type='building_6'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_7'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_8'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_9'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_10'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_11'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_12'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_13'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_14'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_14a'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_15'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_16'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_17'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_18'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_19'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_20'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_21'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_22'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_23'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_24'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_25'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_26'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_27'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_28'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_29'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_30'),
            #self._create_tile(origin_pos=(500,600), tile_type='building_31'),
        ]
        
        # add streets
        # add train (kinematic object)
        # add trainstation
        
        # create people
        self.n_people = n_people
        self.people = [Person(world=self.world,
                              x_init=random.randint(0, screen_size),
                              y_init=random.randint(0, screen_size),
                              collision_radius=2)
                       for i in range(self.n_people)]
        
        # create train
        self.train = Train(world=self.world,
                           movement_start_pos=(70, 5),
                           movement_stop_pos=(130, 270),
                           movement_end_pos=(120, 700),
                           wall_thickness=3)
        
        # setup np-arrays for data
    
    def run(self):
        while self.running:
            
            self.clock.tick(FPS)
            self.world.step(1/FPS)
            
            self.events()            
            self.update()
            if self.running: # and self.render (if rendering is too slow)
                self.draw()
                            
            # save logs
            # evaluate later
    
    def events(self):
        for event in pg.event.get():
            
            if event.type == pg.QUIT:
                pg.quit()
                self.running = False
                
            if event.type == pg.KEYDOWN:
                if event.key == pg.K_ESCAPE:
                    pg.quit()
                    self.running = False
    
    def update(self):
        # stop train at trainstation
        if (pg.time.get_ticks() % 36_000) > 50 and (pg.time.get_ticks() % 9_000) <= 50 \
                        and not self.train.stopped_at_station and self.train.moving:
            self.train.moving = False
            self.train.stopped_at_station = True
            self.train.body.velocity = (0, 0) # stop train
        
        # resume train
        if (pg.time.get_ticks() % 9_000) > 4000 and (pg.time.get_ticks() % 9_000) <= 4050 \
                        and self.train.stopped_at_station and not self.train.moving:
            self.train.moving = True
            self.train.body.velocity = (-1.1, 30)
        
        # respawn train at top
        if (pg.time.get_ticks() % 36_000) <= 50 and self.train.stopped_at_station and self.train.moving:
            self.train.stopped_at_station = False # reset stopped_at_station variable
            self.train.body.position = (70, 5) # respawn train at top
            self.train.body.velocity = (-1.1, 30)
    
    def draw(self):
        # fill the background color
        self.screen.fill(WHITE)
        
        # draw the golm-background image
        golm_img = pg.image.load('images/golm_test.png')
        golm_img = pg.transform.scale(golm_img, (self.screen_size, self.screen_size))
        self.screen.blit(golm_img, (0, 0))
        
        # draw people
        for person in self.people:
            person.draw(self.screen)
        
        # draw train
        self.train.draw(self.screen)
        
        # draw buildings
        for building in self.buildings:
            for wall in building:
                wall.draw(self.screen)
        
        # draw dots for testing
        for i in range(80):
            for j in range(80):
                if i%10 == 0 and j%10 == 0:
                    pg.draw.circle(self.screen, RED, (i*10, j*10), 2) 
                else:
                    pg.draw.circle(self.screen, LIGHT_GREY, (i*10, j*10), 2) 
        
        pg.display.flip() # update entire screen
    
    
    def _create_tile(self, origin_pos, tile_type):
        x, y = origin_pos
        
        if tile_type == 'house':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+80, y)),
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+80)),
                Wall(world=self.world, start_pos=(x+80, y), end_pos=(x+80, y+80)),
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y+80), end_pos=(x+20, y+80)),
                Wall(world=self.world, start_pos=(x+60, y+80), end_pos=(x+80, y+80))
            ]
        # RIGHT-OPEN, LONG
        if tile_type == 'building_1':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+20, y)), # top-right
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+80)), # top-down(left)
                Wall(world=self.world, start_pos=(x, y+80), end_pos=(x+20, y+80)), # bot-right
                # create half-open wall
                Wall(world=self.world, start_pos=(x+20, y), end_pos=(x+20, y+30)), # top-down(right)
                Wall(world=self.world, start_pos=(x+20, y+60), end_pos=(x+20, y+80)),# top-down(right)
            ]
        # RIGHT-OPEN, SMALL SQUARE
        if tile_type == 'building_2':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+30, y)), # top-right
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+30)), # top-down(left)
                Wall(world=self.world, start_pos=(x, y+30), end_pos=(x+30, y+30)), # bot-right
                # create half-open wall
                Wall(world=self.world, start_pos=(x+30, y), end_pos=(x+30, y+10)), # top-down(right)
                Wall(world=self.world, start_pos=(x+30, y+20), end_pos=(x+30, y+30)), # top-down(right)
            ]
        # LEFT-OPEN, LONG
        if tile_type == 'building_3':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+20, y)), # top-right
                Wall(world=self.world, start_pos=(x+20, y), end_pos=(x+20, y+80)), # top-down(right)
                Wall(world=self.world, start_pos=(x, y+80), end_pos=(x+20, y+80)), # bot-right
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+30)), # top-down(left)
                Wall(world=self.world, start_pos=(x, y+60), end_pos=(x, y+80)),# top-down(left)
            ]
        # LEFT-OPEN, MEDIUM
        if tile_type == 'building_4':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+40, y)), # top-right
                Wall(world=self.world, start_pos=(x+40, y), end_pos=(x+40, y+50)), # top-down(right)
                Wall(world=self.world, start_pos=(x, y+50), end_pos=(x+40, y+50)), # bot-right
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+20)), # top-down(left)
                Wall(world=self.world, start_pos=(x, y+40), end_pos=(x, y+50)),# top-down(left)
            ]
        # L-shape
        if tile_type == 'building_5':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+20, y)), # top-right
                Wall(world=self.world, start_pos=(x+20, y), end_pos=(x+20, y+60)), # top-down(right)
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+30)), # top-down(left)
                Wall(world=self.world, start_pos=(x, y+60), end_pos=(x, y+90)),# top-down(left)
                # create right complex
                Wall(world=self.world, start_pos=(x+20, y+60), end_pos=(x+60, y+60)), # bot-right
                Wall(world=self.world, start_pos=(x, y+90), end_pos=(x+60, y+90)), # bot-right
                Wall(world=self.world, start_pos=(x+60, y+60), end_pos=(x+60, y+90)), # top-down(right)
            ]
        # RIGHT-OPEN, SMALL SQUARE
        if tile_type == 'building_6':
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+50, y)), # top-right
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+20)), # top-down(left)
                Wall(world=self.world, start_pos=(x+50, y), end_pos=(x+50, y+20)), # top-down(right)
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y+20), end_pos=(x+20, y+20)), # bot-right
                Wall(world=self.world, start_pos=(x+40, y+20), end_pos=(x+50, y+20)), # bot-right
            ]
        
        return tile_walls

In [11]:
pg.init()
sim = VirusSim(n_people=1000, screen_size=800, world_size=80)
sim.run()